# Exercise 3


Similarly to what we have done in the previous homework sheet, suppose that we have a pipe of infinitesimal thickness. We parametrize the surface of the pipe with two-dimensional polar coordinates $(\theta,z)\in\Omega:=  [0,2\pi)\times(0,1)$, and we study the evolution in the time interval $\mathbb T=[0,T]$ of the temperature of the pipe $u:\mathbb T\times\overline{\Omega}\to\mathbb{R}$.
To do so, this time we will construct a ```full``` discretization, in both time and space.

Consider then the following PDE:

$$
\left\{
\begin{array}{lll}
	\partial_t u = c\left(\frac{1}{r^2}\partial_{\theta}^2u + \partial_z^2u \right) & \text{in }\mathbb{T}\times\Omega, \\
	u(0,\theta,z) = u_0(\theta,z) & \text{for all }(\theta,z)\in\overline{\Omega}, \\
	\partial_zu(t,\theta,0) = \alpha & \text{for all }(t,\theta)\in\overline{\mathbb T}\times[0,2\pi], \\
	\partial_zu(t,\theta,1) = \alpha & \text{for all }(t,\theta)\in\overline{\mathbb T}\times[0,2\pi], \\
	u(t,0,z)=u(t,2\pi,z) & \text{for all }(t,z)\in\overline{\mathbb T}\times[0,1], \\
	\nabla u(t,0,z)=\nabla u(t,2\pi,z) & \text{for all }(t,z)\in\overline{\mathbb T}\times[0,1],
\end{array} \right.
$$
% 
where $r:=\frac{1}{5}>0$ denotes the radius of the pipe, $c:= 10^{-2}>0$ denotes a material property, and $u_0:\overline{\Omega}\to\mathbb{R}$ is the initial condition. Note that we are using periodic boundary conditions with respect to $\theta$, and Neumann boundary conditions with respect to $z$.

For $u_0$ we choose again:
$$
u_0(\theta, z ) := 
\left\{ 
\begin{array}{lcl} 
	1 & \qquad & \text{if} \; 0 \le \theta \le \frac{\pi}{8} \; \text{and} \; \left|z - \frac{1}{2} \right| \le 10^{-1}\\
	0 & \qquad &\text{otherwise}
\end{array}
\right..
$$

<b>Ungraded :</b> Can you describe the physical system, that can be modeled with this kind of boundary conditions ?

### a) 

As usual, write the function ```get_BVP```, that given a size $N\in\mathbb{N}\times\mathbb{N}$ for a uniform discrete grid, and the parameter $\alpha\in\mathbb{R}$, returns the matrix $L_h$, and the vectors $f_h$ and $u_{h,0}$, that represent the space discretization of the PDE:

$$
\left\{
	\begin{array}{ll}
		\partial_tu_h(t) + L_hu_h(t) = f_h & \text{for all }t\in\mathbb T, \\
		u_h(0) = u_{h,0}.
	\end{array}
    \right.
$$


In [1]:
import numpy as np
from scipy.sparse import diags, eye, kron
from scipy.sparse.linalg import spsolve
import matplotlib.pyplot as plt
from scipy.sparse.linalg import expm_multiply


def get_bvp(n,c, r, alpha):
    
    h1 = 2 * np.pi / ( n[0] + 1 )  # step size for theta grid 
    h2 = 1 / ( n[1] + 1 )          # step size for z grid
    
    #Construct T matrix (for theta)
    d0 = - ( 2 / (r**2) / (h1**2) + 1 / (h2**2) ) * np.ones(n[0]+1)                 #main diagonal of T in the boundaries of z
    d1 = - ( 2 / (r**2) / (h1**2) + 2 / (h2**2) ) * np.ones(n[0]+1)                 #main diagonal of T not in the boundaries of z
    d2 = ( 1 / (r**2) / (h1**2) ) * np.ones(n[0])                                   #upper and lower diagonal    
    T1 = diags([d0,d2,d2,1/(r**2)/(h1**2),1/(r**2)/(h1**2)],[0,-1,1,-n[0],n[0]])    #construct T in the boundaries of z
    T2 = diags([d1,d2,d2,1/(r**2)/(h1**2),1/(r**2)/(h1**2)],[0,-1,1,-n[0],n[0]])    #construct T not in the boundaries of z
    
    #Construct L_h matrix
    #For Neumann boundary conditions, I use the central differences to approximate u'(0) and u'(1). So the matrix include two boundaries.
    diags_In1 = np.zeros(n[1]+2)
    diags_In2 = np.ones(n[1]+2)
    diags_In1[0] =1
    diags_In1[-1]=1
    diags_In2[0] =0
    diags_In2[-1]=0
    In1 = diags(diags_In1,0)
    In2 = diags(diags_In2,0)
    T = kron(In1,T1) + kron(In2,T2)  #diagonal T
    D = (n[0]+1) * (n[1]+2)   #size of Lh matrix
    I = ( 1 / (h2**2) ) * ( eye( D,D,-n[0]-1 ) + eye(D,D,n[0]+1) )  #I_N  (this is for z grid part) 
    Lh = - c * ( T + I )
    
    #Construct f_h
    fh = np.zeros( ( D,1 ) )
    fh[: n[0]+1 ] = - alpha / h2  # boundary g0
    fh[ -n[0]-1 :] =  alpha / h2  # boundary g1
    
    #Construct initial u0(theta,z)
    u0 = np.zeros( ( D,1 ) )
    for i in range(n[0]+1):
        for j in range(n[1]+2):
            if i * h1 <= np.pi/8 and abs( j * h2 - 0.5 ) <= 0.1:
                u0[ j * (n[0]+1) + i ] = 1

    return Lh,fh,u0
    

### b)

Write the function ```theta_scheme``` that, given $L_h$, $f_h$, $u_h(t)$, $\text{d}t>0$, and $\theta\in[0,1]$, performs one step of the algorithm with the same name from the lecture, to approximate $u_h(t+\text{d}t)$. 

In [2]:
def theta_scheme( L_h, f_h, u_hk, dt, theta = 0):
    

    b = f_h - ( 1 - theta ) * ( L_h @ u_hk ) + u_hk / dt
    A = diags( np.ones( L_h.shape[0] ), 0 ) / dt + theta * L_h 

    return spsolve(A,b)
      

### c)

We know from the lecture that, in the one-dimensional case, the ratio $\gamma=\text{d}t/h^2$ determines the stability of the discretization method.
With that in mind, let us define the sizes $N_1,N_2,N_3\in\mathbb{N}\times\mathbb{N}$, such that the respective step sizes are $h_1=(1/10,1/14)$, $h_2=(1/20,1/28)$, $h_3=(1/30,1/42)$, and $h_4=(1/40,1/56)$.
	Correspondingly, let us define	
	$$
		\begin{bmatrix}
			\text{d}t_{11} & \text{d}t_{12} & \text{d}t_{13} \\
			\text{d}t_{21} & \text{d}t_{22} & \text{d}t_{23} \\
			\text{d}t_{31} & \text{d}t_{32} & \text{d}t_{33} \\
			\text{d}t_{41} & \text{d}t_{42} & \text{d}t_{43}
		\end{bmatrix}
		=
		\begin{bmatrix}
			1/4 & 1/5 & 1/6 \\
			1/16 & 1/20 & 1/36 \\
			1/36 & 1/45 & 1/54 \\
			1/64 & 1/80 & 1/144
		\end{bmatrix}, \quad\text{i.e.}\quad
		dt_{ij} = \frac{1}{i^2(j+3)},
	$$	
so that the ratio $\gamma_j=\text{d}t_{ij}/h_i^2$ only depends on $j$, and not on $i$.
	
Construct the space discretized problem with size $N_i$ and parameter $\alpha=1$, and
compute the discrete solution at time $T=1$ using the explicit Euler method ($\theta=0$) with time step $\text{d}t_{ij}$, for $i=1,2,3,4$ and $j=1,2,3$. Create a surface plot for every computed solution, and place them on a $4\times 3$ grid ( use ```matplotlib.pyplot.subplots```).

In [10]:
%matplotlib qt
T = 1.
n = (20,30)
r = 1/5
c = 1e-2
alpha = 1
fig = plt.figure(figsize=plt.figaspect(0.5))
q = 1
for i in range(4):
    for j in range(3):
        dt = 1/ ( (i+1)**2 ) / ( j + 1 + 3 )  # dt_ij
        N  = ( (i + 1) * 10 - 1, (i + 1) * 14 - 1 ) # get N from h=1/(N+1)  
        n1 = N[0]
        n2 = N[1]
        Lh,fh,uh = get_bvp( N, c, r, alpha)  # get L_h,f_h,u_0 from get_bvp for every N 
          
        t = 0    
        while t < T:   
            uh = theta_scheme( Lh, fh, uh, dt, 0)
            uh = uh.reshape(-1,1)
            t  = t + dt
        

        theta = np.linspace( 0, 2*np.pi, n1 + 1 )  # theta grid
        z     = np.linspace( 0, 1, n2+2)           # z grid
        Uh    = np.zeros( ( n1+1, n2+2 ) )                 #Construct matrix u_h
        for p in range( n1+1 ):
            for w in range( n2+2 ):
                Uh[p][w] = uh[ w * n1 + p ]
                
        z, theta = np.meshgrid(z, theta)
        ax = fig.add_subplot(4, 3, q, projection='3d')
        ax.plot_surface(z, theta, Uh,cmap='viridis', edgecolor='none')
      
        
        ax.set_title('t=1/{}'.format( ( (i+1)**2 ) * ( j + 1 + 3 ) ))
     
        ax.set_zlabel('u_h')
        plt.show()
        
        q += 1     

### d)

Let now $\alpha = 0$, and consider again the following functional:

$$
H_h(t) := \frac{2 \pi } { n_1 n_2 } \sum_{i,j} u(x_{i,j})^2.
$$
Define the size $N\in\mathbb{N}\times\mathbb{N}$, such that the step size is $h=(1/20,1/30)$, and compute the discrete solutions $u_h^k$ for $t=[0,1]$, using the theta scheme with $\theta \in \left\{  0, \frac{1}{2}, 1 \right\}$ and $\text{d}t\in\{0.05,0.02\}$, as well as the exact solution $u_h$ (see previous homework sheets).
	Compute the value of the functional $H_h$ along every solution, then create a different figure for each choice of $\theta$.
	Compare in each figure the evolution in time of $H_h(t)$, for the two discrete solutions $u_h^k$ obtained with different time steps, and for the exact solution $u_h$.
    
<b>Suggestion:</b> choose a fine time grid to plot $H_h(t)$ for the exact solution of the ODE, so it will look smoother.

In [7]:
def H_h(n, t,dt, theta ):
    
    Lh,fh,uh = get_bvp( n, 1e-2, 1/5, 0 ) # get L_h, f_h, u_0
    
    T = 0    
    while T < t:    # get u_h(t) 
        uh = theta_scheme( Lh, fh, uh, dt, theta)
        uh = uh.reshape(-1,1)
        T  = T + dt

    H = 2 * np.pi * sum([i**2 for i in uh])/n1/n2 # get H
    return float(H)

def H_exact( n, t):
    
    Lh,fh,uh = get_bvp( n, 1e-2, 0.2, 0)
    
    Lht = -Lh * t
    uh = expm_multiply( Lht, uh ) # Solve e^(-Lt)u_0 = 0
    H = 2 * np.pi * sum([i**2 for i in uh])/n1/n2 # get H
    
    return float(H)

n = (19,29) # h = 1/(N+1)
plt.figure(figsize=(8,4))
plt.suptitle('Evolution in time of H(t)')
plt.subplots_adjust(left=0.125, bottom=0.4, right=0.9, top=0.7, wspace=0.4, hspace=0.2)
for Theta in [0,0.5,1]:
    plt.subplot(1,3,[0,0.5,1].index(Theta)+1)
    Hlist1 = []
    Hlist2 = []
    Hlist3 = []
    for i in np.arange(0,1.05,0.05):
        Hlist1.append(H_h((19,29),i,0.05,Theta))
    plt.plot(np.arange(0,1.05,0.05),Hlist1,label='t=0.05',color='limegreen',linewidth=0.7,linestyle='--')
    for j in np.arange(0,1.02,0.02):
        Hlist2.append(H_h((19,29),j,0.02,Theta))
    plt.plot(np.arange(0,1.02,0.02),Hlist2,label='t=0.02',color='slateblue',linewidth=1,linestyle='-.')
    for k in np.arange(0,1.01,0.01):
        Hlist3.append(H_exact((19,29), k))
    plt.plot(np.arange(0,1.01,0.01),Hlist3,label='exact solution',color='brown',linewidth=0.5)
    plt.xlabel('time')
    plt.ylabel('H(t)')
    plt.title('theta={}'.format(Theta))
    plt.rcParams.update({'font.size':5.5})
    plt.legend()

        